In [36]:
# installing openai 0.28 # openai 1.0.0
!pip install numpy openai==0.28  tiktoken gradio
# load data from sample_data directory
import os
import csv
import json
import numpy as np
from collections import defaultdict
import tiktoken # for token
import gradio as gr


In [3]:
# locate data of CSV type

ashu_data = "/content/sample_data/hotel_hospitality_data.csv"
ashu_clean_data = []
# opening file and check data
with open(ashu_data,'r',encoding='utf-8-sig') as f1:
  ashu_csv_reader = csv.reader(f1)
  for i in ashu_csv_reader:
    for j in i:
      try:
        # replace brackets , double quotes ,
        replace_data1 = j.replace('["','').replace('"]','').replace('\"','"')
        # load each object data in json context
        replace_data1_json = json.loads(replace_data1)
        # append data in a list for futher saving
        ashu_clean_data.append(replace_data1_json)
      except json.JSONDecodeError as e:
        print("some error found")

# lets printing clean data
print(ashu_clean_data[0])

# save this data for reuse
ashu_data_jsonl = "/content/sample_data/ashu_hotel_hospitality_data.jsonl"
# using file handling
with open(ashu_data_jsonl,'w',encoding='utf-8') as f2:
  for items in ashu_clean_data:
    f2.write(json.dumps(items) + '\n')


{'messages': [{'role': 'system', 'content': "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."}, {'role': 'user', 'content': 'Where is the gym located?'}, {'role': 'assistant', 'content': "Our gym is located on the 2nd floor. I'll be happy to show you the way if you need help!"}]}


In [7]:
# printing data for validataion purpose
with open(ashu_data_jsonl,'r',encoding='utf-8') as f3:
  ashu_datasets = [ json.loads(i) for i in f3]

# total length
print("length of data dataset is ",len(ashu_datasets))
print("first example ")
for ashud in ashu_datasets[0]['messages']:
  print(ashud)


length of data dataset is  18
first example 
{'role': 'system', 'content': "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."}
{'role': 'user', 'content': 'Where is the gym located?'}
{'role': 'assistant', 'content': "Our gym is located on the 2nd floor. I'll be happy to show you the way if you need help!"}


In [8]:
#**format validation
# Format error checks
format_errors = defaultdict(int)

for ex in ashu_datasets:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [13]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [15]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in ashu_datasets:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
#n_too_long = sum(l > 16,385 for l in convo_lens)
#print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 68, 91
mean / median: 76.38888888888889, 74.0
p5 / p95: 71.4, 87.5

#### Distribution of num_assistant_tokens_per_example:
min / max: 16, 39
mean / median: 24.444444444444443, 23.0
p5 / p95: 19.0, 35.5


In [21]:
#UPloading data to gpt model
import openai
# loading api keys
openai.api_key = ""

In [17]:
# please don't upload the same data
# # uploading jsonl data to gpt api server
# training_data = "/content/sample_data/ashu_hotel_hospitality_data.jsonl"
# # creting openai method for uplaod
# training_response = openai.File.create(file=open(training_data,'rb'), purpose="fine-tune")
# # print response
# print(training_response)
# # print file id which we are uploading
# ashu_file_id = training_response["id"]
# print("ashu uploaded file id is ",ashu_file_id)

In [18]:
# # dont not perform
# # creating fine tuning job
# ashu_fm_suffix = "ashu_hotel_bot"
# # creting fine tune arguments
# fineTune_response = openai.FineTuningJob.create(
#     training_file=ashu_file_id,
#     suffix=ashu_fm_suffix,
#     model="gpt-3.5-turbo")

# # print id
# ashu_job_id=fineTune_response["id"]

# print(fineTune_response)


In [20]:
# # list training job events
# #list events as fine-tuning progresses
# response = openai.FineTuningJob.list_events(id=ashu_job_id, limit=50)

# events = response["data"]
# events.reverse()

# for event in events:
#     print(event["message"])

In [26]:
# we can retive existing fine tuned model ids
myaccount_models = openai.Model.list()
#print(myaccount_models['data'])
# listing it using loop
for model in myaccount_models['data']:
  print(model['id'])

dall-e-3
gpt-4-1106-preview
dall-e-2
tts-1-hd-1106
tts-1-hd
text-embedding-3-large
gpt-4-0125-preview
babbage-002
gpt-4-turbo-preview
gpt-4o
gpt-4o-2024-05-13
text-embedding-3-small
tts-1
gpt-3.5-turbo
whisper-1
gpt-4o-2024-08-06
text-embedding-ada-002
gpt-3.5-turbo-16k
davinci-002
gpt-4-turbo-2024-04-09
tts-1-1106
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-3.5-turbo-1106
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4-0613
gpt-4
ft:gpt-3.5-turbo-0125:personal:ashu-walmartbot:9tVvJxZA:ckpt-step-54
ft:gpt-3.5-turbo-0125:personal:ashu-walmartbot:9tVvJe6E:ckpt-step-72
ft:gpt-3.5-turbo-0125:personal:ashu-walmartbot:9tVvJ5oT
ft:gpt-3.5-turbo-0125:personal:samantha-test:9tW4MOZd
ft:gpt-3.5-turbo-0125:personal:samantha-test:9tW4MFcG:ckpt-step-114
ft:gpt-3.5-turbo-0125:personal:samantha-test:9tW4MwCX:ckpt-step-57
ft:gpt-3.5-turbo-0125:personal:hotel-bot:9teGNfPK:ckpt-step-54
ft:gpt-3.5-turbo-0125:personal:hotel-bot:9teGNgSX:ckpt-step-72
ft:gpt-3.5-turbo-0125

In [33]:
# creating data for input purpose for testing the fine tuned model
ashu_test_data = []
system_message = "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."
user_input = "hey i am willing to park my BMW car please suggest some good and safe options"

# creating message format
ashu_test_data.append({ "role": "system" , "content" : system_message})
# adding user input

ashu_test_data.append({ "role": "user" , "content" : user_input})
# print user input to test
print(ashu_test_data)

[{'role': 'system', 'content': "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."}, {'role': 'user', 'content': 'hey i am willing to park my BMW car please suggest some good and safe options'}]


In [37]:
# calling fine model on hotel hospitality data to get response
ashu_response = openai.ChatCompletion.create(
    model="ft:gpt-3.5-turbo-0125:personal:ashu-walmartbot:9tVvJxZA:ckpt-step-54",
    messages=ashu_test_data,
    max_tokens=500,
    temperature=0
)
# lets print response details
print(ashu_response['choices'][0]['message']['content'])

Absolutely! We have secure parking options available. You can park your BMW in our designated parking area. I'll guide you there.


In [38]:
# creating GUI internface
def ashu_get_response(user_input):
  ashu_test_data = []
  system_message = "You are an overly friendly hospitality chatbot named Chatner who just loves to help people, and you're not satisfied unless the customer is completely satisfied."
  # creating message format
  ashu_test_data.append({ "role": "system" , "content" : system_message})
  # adding user input

  ashu_test_data.append({ "role": "user" , "content" : user_input})

  ashu_response = openai.ChatCompletion.create(
    model="ft:gpt-3.5-turbo-0125:personal:ashu-walmartbot:9tVvJxZA:ckpt-step-54",
    messages=ashu_test_data,
    max_tokens=500,
    temperature=0
)
  return ashu_response['choices'][0]['message']['content']

# creating gui
# Set up the Gradio interface
iface = gr.Interface(
    fn=ashu_get_response,
    inputs=gr.Textbox(label="Enter your question:"),
    outputs=gr.Textbox(label="Response: "),
    title="Chatbot Interface",
    description="Ask questions to the friendly hospitality chatbot."
)

# Launch the interface
iface.launch()




Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://50f50f5f9319162ad6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
